In [1]:
import cv2
import mediapipe as mp
import math
import serial
import time
import tkinter as tk
from PIL import Image, ImageTk

# Initialize Serial Communication with Arduino
arduino = serial.Serial(port='COM22', baudrate=9600, timeout=1)
time.sleep(2)  # Wait for the serial connection to initialize

# Initialize Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Store last valid frequency
last_valid_frequency = 0


# Function to detect individual fingers (1 for up, 0 for down)
def detect_fingers(hand_landmarks):
    finger_tips = [8, 12, 16, 20]  # Index, Middle, Ring, Pinky
    thumb_tip = 4
    finger_states = [0, 0, 0, 0, 0]  # Thumb, Index, Middle, Ring, Pinky

    # Check thumb
    if hand_landmarks.landmark[thumb_tip].x < hand_landmarks.landmark[thumb_tip - 1].x:
        finger_states[0] = 1  # Thumb is up

    # Check the other fingers
    for idx, tip in enumerate(finger_tips):
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y:
            finger_states[idx + 1] = 1  # Other fingers are up

    return finger_states


# Function to calculate hand rotation angle (yaw) and map it to frequency
def get_rotation_frequency(hand_landmarks):
    wrist = hand_landmarks.landmark[0]  # Wrist landmark
    index_base = hand_landmarks.landmark[5]  # Base of index finger

    # Calculate angle using inverse tangent (atan2)
    delta_x = index_base.x - wrist.x
    delta_y = index_base.y - wrist.y
    angle = math.degrees(math.atan2(delta_y, delta_x))  # Angle in degrees

    # Map angle (-90 to +90) to frequency range (e.g., 100 Hz to 1000 Hz)
    fOut = int((angle + 90) / 180 * (1000 - 100) + 100)  # Scale to range 100-1000 Hz
    return fOut


# Initialize Tkinter GUI
root = tk.Tk()
root.title("Hand Gesture Control")
root.geometry("800x600")

# GUI Labels
video_label = tk.Label(root)
video_label.pack()

finger_label = tk.Label(root, text="Finger States: ", font=("Arial", 14))
finger_label.pack()

frequency_label = tk.Label(root, text="Rotation Frequency: ", font=("Arial", 14))
frequency_label.pack()

# Start capturing video
cap = cv2.VideoCapture(0)


def update_frame():
    global last_valid_frequency

    success, image = cap.read()
    if not success:
        return

    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Detect fingers and hand rotation
            fingers_state = detect_fingers(hand_landmarks)

            # Check if all fingers are closed
            if sum(fingers_state) == 0:  # All fingers down
                fOut = last_valid_frequency  # Lock the last known frequency
            else:
                fOut = get_rotation_frequency(hand_landmarks)  # Compute frequency
                last_valid_frequency = fOut  # Update last valid frequency

            # Send data to Arduino (Frequency + Fingers State)
            data_to_send = f"{fOut},{''.join(map(str, fingers_state))}\n"
            arduino.write(data_to_send.encode())  # Send via Serial

            # Update GUI Labels
            finger_label.config(text=f"Finger States: {fingers_state}")
            frequency_label.config(text=f"Rotation Frequency: {fOut} Hz")

    # Convert image for Tkinter display
    img = Image.fromarray(image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)

    root.after(10, update_frame)  # Update every 10ms


# Start GUI loop
update_frame()
root.mainloop()

# Release video capture and close serial port when GUI is closed
cap.release()
arduino.close()

SerialException: could not open port 'COM22': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)